Scraping, and Cleaning FBS Coach Salary Data with Python
======================================
 
The purpose of this notebook is to show the process behind scraping and cleaning the data used in my blog post "FBS Football Coach Salaries". Since this is a Jupyter notebook, you can follow along and execute the code yourself, which will help you better understand the process. 

For scraping, I used the Python libraries requests and BeautifulSoup. For cleaning and structuring, I used pandas and numpy.

## Scraping the Data

Since 2014, USA Today has requested FBS football coach compensation data from all 128 FBS schools. Each year, they update their salary data pages, in "data links" below. The catch here was that USA Today doesn't keep the past years' data live on their website, and there was no easily accessible archive (easily accessible means on the first couple pages of Google). Luckily, the [Wayback Machine](http://archive.org/web/) contains archives of the USA Today salary data. If you haven't ever used the Wayback Machine, I encourage you to check it out. To get the old versions of the data, I simply copied and pasted the appropriate links into the Wayback machine, then accessed the page matching that year's assistant coach methodology publish date. 

The methodologies for gathering compensation data, notes about abnormal data, and  the explanations for each type of compensation received, can be found at the "methodology links" below. 
#### Data links 
[Head coaches](http://sports.usatoday.com/ncaa/salaries/) 
[Assistant coaches](http://sports.usatoday.com/ncaa/salaries/football/assistant) 
[Strength coaches](http://sports.usatoday.com/ncaa/salaries/football/strength) 
#### Methodology Links

2014 Methodologies
[Head coaches](http://sports.usatoday.com/2014/11/19/2014-ncaa-football-head-coach-salaries-methodology/)
[Assistants](http://sports.usatoday.com/2014/12/10/2014-ncaa-football-assistant-coaches-methodology/)

2015 Methodologies
[Head coaches](http://sports.usatoday.com/2015/10/08/2015-ncaa-football-head-coach-salaries-methodology/)
[Assistants](http://sports.usatoday.com/2015/12/08/2015-ncaa-football-assistant-coach-salaries-methodology/)

2016 Methodologies
[Head coaches](http://sports.usatoday.com/2016/10/25/2016-ncaa-football-head-coach-salaries-methodology/)
[Assistants](http://sports.usatoday.com/2016/12/06/2016-ncaa-football-assistant-coach-salaries-methodology/)
[Strength coaches](http://sports.usatoday.com/2016/12/06/2016-ncaa-football-strength-coach-salaries-methodology/)

2017 Methodologies
[Head coaches](http://sports.usatoday.com/2017/10/25/2017-ncaa-football-head-coach-salaries-methodology/)
[Assistants](http://sports.usatoday.com/2017/12/06/2017-ncaa-football-assistant-coach-salaries-methodology/)
[Strength coaches](http://sports.usatoday.com/2017/12/06/2017-ncaa-football-strength-coach-salaries-methodology/)

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats, integrate
import seaborn as sns
from ggplot import *


/Users/rachelehlers/anaconda3/lib/python3.6/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/Users/rachelehlers/anaconda3/lib/python3.6/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/Users/rachelehlers/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [73]:
def makedataframe(link): 
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'lxml')
    table = soup.find_all('table')
    data = pd.read_html(str(table))
    # ifelse to handle the waybackmachine's different table output
    if len(data) > 1:
        data = data[0].join(data[1], how = 'outer', on = None )
    else:
        data = data[0]
    data = data.replace('--', np.NaN) # null values in the table are '--'
    return data

In [136]:
###  Head coaches ###
headcoachlink = 'http://sports.usatoday.com/ncaa/salaries/'

fourteen = makedataframe('https://web.archive.org/web/20141230041628/' + 
                         headcoachlink)
fifteen = makedataframe('https://web.archive.org/web/20151214221657/' + 
                        headcoachlink)
sixteen = makedataframe('https://web.archive.org/web/20161215053646/' + 
                        headcoachlink)[:-1]
seventeen = makedataframe(headcoachlink)[:-1] # Remove bottom column labels
######################


###  Assistant coaches ###
assistantlink = 'http://sports.usatoday.com/ncaa/salaries/football/assistant'
astfourteen = makedataframe('https://web.archive.org/web/20150129211154/' + 
                            assistantlink)
astfifteen = makedataframe('https://web.archive.org/web/20160130233153/' + 
                           assistantlink)
astsixteen = makedataframe('https://web.archive.org/web/20170130233153/' + 
                           assistantlink)[:-1]

astseventeen = makedataframe(assistantlink)[:-1] # Remove bottom column labels
 
###########################

### Strength coaches ###
strengthcoachlink = 'http://sports.usatoday.com/ncaa/salaries/football/strength'
strengthsixteen = makedataframe('https://web.archive.org/web/20170130233153/' + strengthcoachlink)
strengthseventeen = makedataframe(strengthcoachlink)[:-1] # Remove bottom column labels
########################

## Data Cleaning Process
When comparing data between multiple datasets, consistency is imperative. Thus, ensuring similarity between sets with data types, labels, null values, and anywhere else will be referenced in making computations. 


- uniformity of column names
- appropriate types
- deal with null values
- adding values where appropriate


### Uniformity of column names
Columns in pandas dataframes are referred to by string (I show in a later steps how to refer to different components of dataframes). Let's look at a couple sets of column names from head coach data. 

In [150]:
print("2014",fourteen.columns)
print("2015",fifteen.columns)
print("2016",sixteen.columns)
print("2017",seventeen.columns)

2016 Index(['Rk', 'School', 'Conf', 'Coach', 'School Pay', 'Total Pay', 'Max Bonus',
       'Asst Pay Total', 'School Buyout AS OF 12/1/17'],
      dtype='object')


Hopefully it's clear that comparing these dataframes would be difficult with the given column names. Since few of the tables have the same column titles and order, I was forced to manually rename each set of columns. I converted each column name to a shorter, more sensible name, retaining similarity between datasets where necessary.

In [151]:
fourteen.columns = ['rank', 'school', 'conf', 'headcoach', 'schoolpay',
                    'otherpay', 'totalpay', 'maxbonus', 'stafftotal']
fifteen.columns = ['rank', 'school', 'conf', 'headcoach', 'schoolpay',
                   'otherpay', 'totalpay', 'maxbonus', 'bonusespaid',
                   'stafftotal']
sixteen.columns = ['rank', 'school', 'conf', 'headcoach', 'schoolpay',
                   'otherpay', 'totalpay', 'maxbonus', 'bonusespaid',
                   'stafftotal']
seventeen.columns = ['rank', 'school', 'conf', 'headcoach', 'schoolpay',
                     'totalpay', 'maxbonus', 'bonusespaid', 'stafftotal', 
                     'buyout']



astfourteen.columns = ['rank', 'school', 'conf', 'astcoach', 'schoolpay',
                       'otherpay', 'totalpay', 'maxbonus', 'stafftotal']
astfifteen.columns = ['rank', 'school', 'conf', 'astcoach', 'schoolpay',
                      'otherpay', 'totalpay', 'maxbonus', 'stafftotal'] 
astsixteen.columns = ['rank', 'school', 'conf', 'astcoach', 'schoolpay',
                      'otherpay','totalpay', 'maxbonus', 'stafftotal']
astseventeen.columns = ['rank', 'school', 'conf', 'astcoach', 'schoolpay',
                      'totalpay', 'maxbonus', 'stafftotal','buyout']


strengthsixteen.columns = ['rank', 'school', 'conf', 'strengthcoach', 'schoolpay',
                      'otherpay','totalpay', 'maxbonus', 'stafftotal']
strengthseventeen.columns = ['rank', 'school', 'conf', 'strengthcoach', 'schoolpay',
                           'totalpay', 'maxbonus' ,'stafftotal', 'buyout'] 


### Referencing items in a Pandas dataframe 
Now, the column names are shorter, cleaner, and consistent(where applicable) across datasets, let's go through a few different examples to show you how to reference columns and their items. 

#### Referencing a dataframe
First, start by printing the assistant coach data from 2014.

In [152]:
astfourteen

,rank,school,conf,astcoach,schoolpay,otherpay,totalpay,maxbonus,stafftotal
0,1,Virginia Tech,ACC,Bud Foster,"$1,356,500","$13,000","$1,369,500","$82,500","$3,583,250"
1,2,Alabama,SEC,Kirby Smart,"$1,350,000","$5,850","$1,355,850","$250,000","$5,213,400"
2,3,LSU,SEC,Cam Cameron,"$1,300,000","$15,967","$1,315,967","$115,000","$5,499,269"
3,4,LSU,SEC,John Chavis,"$1,300,000","$12,300","$1,312,300","$115,000","$5,499,269"
4,5,Clemson,ACC,Chad Morris,"$1,300,000","$1,250","$1,301,250","$120,417","$4,448,225"
5,6,Louisville,ACC,Todd Grantham,"$975,000",NaN,"$975,000",$0,"$3,225,000"
6,7,Michigan State,Big Ten,Pat Narduzzi,"$905,033","$2,000","$907,033",$0,"$3,205,702"
7,8,Arizona State,PAC-12,Mike Norvell,"$900,000","$1,560","$901,560","$490,000","$3,111,620"
8,9,Baylor,Big 12,Phil Bennett,"$897,926",NaN,"$897,926",NaN,NaN
9,10,Michigan,Big Ten,Greg Mattison,"$888,333",$0,"$888,333","$150,000","$3,504,323"


#### Referencing a single column
Now, let's look at a single column in a dataframe. The following code will print the total pay for assistant coaches in 2014.

In [210]:
# Print column of dataframe.
#astfourteen['schoolpay']

# Print rows of dataframe. Includes first, excludes last.
#astfourteen['schoolpay'][5:10]

# Print whole dataframe.
#astfourteen

5    975000.0
6    905033.0
7    900000.0
8    897926.0
9    888333.0
Name: schoolpay, dtype: float64

#### Referencing one entry in a column
Now, run the following code to print the 6th entry of the column we printed above. Recall that indexes start at 0.

In [154]:
astfourteen['totalpay'][5]

'$975,000'

#### Referencing a range of entries in a column
The method of printing a series of entries in a column is similar to printing one entry. The only difference is that the index should reference a range of values. The following code will print the 6th through 10th entries of the 2014 assistant coach total pay column.

In [155]:
astfourteen['totalpay'][5:10]

5    $975,000
6    $907,033
7    $901,560
8    $897,926
9    $888,333
Name: totalpay, dtype: object

### Stripping appropriate characters
It's clear that the monetary values are strings, because their formats include commas and dollar signs. To convert the values to floats, the extra characters must first be stripped. In the following step, I initialize a list of the dataframes to iterate through during the cleaning process. Then, the characters are stripped from the monetary data. 

In [1]:
#list of dataframes for reference in loops
dflist = [fourteen, fifteen, sixteen, seventeen, astfourteen, astfifteen, 
          astsixteen, astseventeen, strengthsixteen, strengthseventeen]

for item in dflist:
    for x in item.columns:
        item[x] = item[x].astype(str).str.replace('$', '')
        item[x] = item[x].astype(str).str.replace(',', '')

NameError: name 'fourteen' is not defined

Now, let's call the same command where a series of entries was referenced. Re-run the second-to-last codeblock to see the change. Now, this doesn't mean the numbers are ready for computation. Let's see what happens when we add two items from the output. 

In [159]:
astfourteen['totalpay'][5] + astfourteen['totalpay'][6]

1882033.0

We know that `975000 + 907033` should not yield `975000907033`. This was the resulting output because the values are still in string form.

### Type Coercion
The numbers are still in string form and must be converted to floats before any quantitative analysis is performed. This can be accomplished with the `pd.to_numeric` command. Below, I initialize a list of possible numeric column names from all dataframes. Then I loop through each dataframe's columns, changing their values to floats if their name is in the list.

In [160]:
numericcolumns = ['rank', 'schoolpay', 'otherpay', 'totalpay', 'maxbonus', 
                  'bonusespaid', 'stafftotal']

for item in dflist:    
    for x in item.columns: 
        if x in numericcolumns:
            item[x] = pd.to_numeric(item[x], errors = 'coerce')

Now, re-run the second-to-last cell to see what happens when the values are added after changing their type. You should see the sum of the values `975000.0` and `907033.0`, which is `1882033.0`.

### Adding new columns
In the following, I will add two new columns to each dataframe. The `proportion` variable will equal each coach's school salary divided by the staff total. A variable called `category` will be added to each row, labeled `Power Five` if the coach's school is in a Power Five conference and `Others` if the team belongs to any of the other conferences in the FBS.


In [165]:
powerfive = ['ACC', 'Big 12', 'Big Ten', 'Pac-12', 'SEC']

for item in dflist: 
    item['category'] = ['Power Five' if x in powerfive else 'Others' for x in item['conf']]
    item['proportion'] = item['schoolpay']/item['stafftotal']